In [1]:
import keras
import json
from datetime import datetime
import numpy as np
import pandas as pd

Using TensorFlow backend.


In [2]:
keras.__version__

'2.2.4'

In [3]:
filename = './input/movie_conversations.txt'
conversation = []
with open(filename, 'r') as f:
    for line in f.readlines():
        conversation.append(line.split(' +++$+++ ')[-1])

In [4]:
filename = './input/movie_lines.txt'
id_to_line = {}
with open(filename, 'r') as f:
    for line in f.readlines():
        tmp = line.strip().split(' +++$+++ ')
        if tmp[0][0] != 'L':
            id_to_line[tmp[0][1:]] = tmp[-1]
        else:
            id_to_line[tmp[0]] = tmp[-1]

In [5]:
print(len(conversation))
print(len(id_to_line))

83097
304713


In [6]:
raw_all_text = []
for lines in conversation:
    content = [val[1:-1] for val in lines.strip()[1:-1].split(', ')]
    tmp = []
    for val in content:
        try:
            tmp.append(id_to_line[val])
        except:
            print("line {} doesn't exist.".format(val))
    raw_all_text.append(tmp)

In [7]:
import re
def clean_text(text):
    text = text.lower()
    text = re.sub(r"\'m", " am", text)
    text = re.sub(r"\'s", " is", text)
    text = re.sub(r"\'re", " are", text)
    text = re.sub(r"\'ll", " will", text)
    text = re.sub(r"\'ve", " have", text)
    text = re.sub(r"\'re", " are", text)
    text = re.sub(r"\'d", " would", text)
    text = re.sub(r"won't", "will not", text)
    text = re.sub(r"can't", "cannot", text)
    text = re.sub(r"n't", " not", text)
    text = re.sub(r"[-()\"#/@;:<>{}`+=~|.!?,$%&/\\\t\n]", "", text)
    
    #!\"#$%&()*+,-./:;<=>?@[\\]^_`{\|}~\t\n
    return text

In [8]:
clean_all_text = []
for text in raw_all_text:
    tmp = []
    for val in text:
        tmp.append(clean_text(val))
    clean_all_text.append(tmp)

In [9]:
clean_input_text = []
clean_target_text = []
for val in clean_all_text:
    if len(val) % 2 == 1:
        val = val[:-1]
    for i, line in enumerate(val):
        if i % 2 == 0:
            clean_input_text.append(line)
        else:
            clean_target_text.append(line)

In [10]:
print(clean_input_text[0])
print(clean_target_text[0])
print(clean_input_text[1])
print(clean_target_text[1])

can we make this quick  roxanne korrine and andrew barrett are having an incredibly horrendous public break up on the quad  again
well i thought we would start with pronunciation if that is okay with you
not the hacking and gagging and spitting part  please
okay then how 'bout we try out some french cuisine  saturday  night


In [11]:
all_input_length = [len(val.split(' ')) for val in clean_input_text]
ave_input_length = np.average(all_input_length)
max_input_length = np.max(all_input_length)
min_input_length = np.min(all_input_length)
print(max_input_length)
print(ave_input_length)
print(min_input_length)

654
11.537495927896623
1


In [12]:
all_target_length = [len(val.split(' ')) for val in clean_target_text]
ave_target_length = np.average(all_target_length)
max_target_length = np.max(all_target_length)
min_target_length = np.min(all_target_length)
print(max_target_length)
print(ave_target_length)
print(min_target_length)

1128
10.96952980779672
1


In [13]:
print(len(clean_input_text))
print(len(clean_target_text))

138135
138135


In [14]:
min_length = 2
max_length = 20
encode_text = []
decode_text = []
for que, ans in zip(clean_input_text, clean_target_text):
    que_len = len(que.split(' '))
    ans_len = len(ans.split(' '))
    if que_len < min_length or que_len > max_length or ans_len < min_length or ans_len > max_length - 2:
        continue
    else:
        encode_text.append(que)
        decode_text.append('<BOS> '+ ans + ' <EOS>')

In [15]:
print(len(encode_text))
print(len(decode_text))

83073
83073


In [16]:
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer

In [17]:
VOC_SIZE = 10000
tokenizer = Tokenizer(filters='!"#$%&()*+,-./:;=?@[\\]^_`{|}~\t\n', num_words=VOC_SIZE)
tokenizer.fit_on_texts(encode_text + decode_text)

In [18]:
sequence_encode_text = tokenizer.texts_to_sequences(encode_text)
sequence_decode_text = tokenizer.texts_to_sequences(decode_text)

In [19]:
pad_encode_text = pad_sequences(sequence_encode_text, maxlen=max_length, padding="post")
pad_decode_text = pad_sequences(sequence_decode_text, maxlen=max_length, padding="post")

In [20]:
import pickle

In [23]:
print(pad_encode_text[0])

[   7    6   21   21 5879  427  135    0    0    0    0    0    0    0
    0    0    0    0    0    0]


In [32]:
with open('pad_encode_text.pk', 'wb') as f:
    pickle.dump(pad_encode_text, f)
with open('pad_decode_text.pk', 'wb') as f:
    pickle.dump(pad_decode_text, f)

In [33]:
with open('tokenizer.pk', 'wb') as f:
    pickle.dump(tokenizer, f)